In [2]:
from postprocessing import *
import random, copy
import numpy as np
import time, os, random, shutil, pickle, ray
# from cgp import *
import matplotlib.pyplot as plt
from configuration import config
import warnings
warnings.filterwarnings('ignore')

In [71]:
def rollout(dataset, policy):
    x, y, z = dataset
    count = 0
    for xi,yi,zi in zip(x,y,z):
        # input = [d[0], d[1]]
        # label = d[2]
        input = [xi, yi]
        output = policy.eval(*input)
        if type(output)!=float:
            continue
        # print(zi, output, zi == output)
        # if zi == output:
        #     count += 1
        count += np.abs(zi-output)
    return count / len(x)

def rollout2(dataset, policy):
    x, y = dataset
    z = [xx>=1.1*yy for xx,yy in zip(x,y)]
    count = 0
    for xi, yi, zi in zip(x,y,z):
        input = [xi, yi]
        output = policy.eval(*input)
        if output == zi:
            count += 1
    return count / 1000

def rollout3(dataset, policy):
    train, label = dataset
    count = 0
    for xyz, l in zip(train, label):
        output = policy.eval(*xyz)
        
        if not type(output)==np.ndarray:
            # print(output, l)
            if output == l:
                count += 1
    return count / 100

def rollout4(dataset, policy):
    train, label = dataset
    count = 0
    for data, l in zip(train, label):
        quantity, price = policy.eval(*data)
        if not type(quantity)==np.ndarray:
            if not type(price)==np.ndarray:
                # print(output, l)
                if quantity == l[0] and price == l[1]:
                    count += 1
    return count / len(label)

def rollout5(dataset, policy):
    train, label = dataset
    count = 0
    for data, l in zip(train, label):
        side, quantity, price = policy.eval(*data)
        if (side==l[0]).all() and (quantity==l[1]).all() and (price==l[2]).all():
            count += 1
    return count / len(label)

In [72]:
def get_data():
    num = 100
    dim = 10
    x, y, z = [], [], []
    for i in range(num):
        xi = np.random.rand(dim,)
        x.append(xi)
        yi = xi * (np.random.uniform([-0.15]*dim, [0.15]*dim) + 1)
        y.append(yi)
        
        zi = np.sum(np.sign(np.maximum((yi-xi)/xi-0.1, 0))*xi)
        z.append(zi)
        
    return [x, y, z]

def get_data2():
    num = 1000
    train, label = [], []
    for i in range(num):
        x = np.random.uniform(-100,100)
        y = x * (1+np.random.uniform(-0.2, 0.2))
        train.append(x)
        label.append(y)
    return [train, label]

def get_data3():
    num = 100
    train, label = [], []
    for i in range(num):
        x = np.random.randint([-20]*10, [20]*10)
        y = np.random.randint(-20, 20)
        z = np.random.randint([0]*10, [100]*10)
        train.append((x,y,z))
        label.append(z[x>=y].sum())
    return [train, label]

def get_data4():
    '''
    将“涨停检测器”和“三元求和器”拼起来
    '''
    num = 100
    train, label = [], []
    for i in range(num):
        quote_price = np.random.randint(10, 100)
        # quote_quantity = np.random.randint
        open_price = np.random.uniform(0.8, 1.2) * quote_price
        bid_price = np.random.randint([1]*5, [150]*5)
        bid_quantity = np.random.randint([1]*5, [20]*5)
        
        if quote_price >= int(1.1 * open_price): # 涨停
            if quote_price <= bid_price.max(): # 小于买单里。， 的最高价，有可能成交
                index = bid_price >= quote_price # ,1.1*open_price
                quantity = bid_quantity[index].sum()
            else: quantity = 0
        else: quantity = 0
        
        train.append((quote_price, open_price, bid_price, bid_quantity))
        label.append(quantity)
    return [train, label]
def get_data5():
    '''
    手工设计的数据集
    '''
    train = [
        # qoute_p > 1.1*open_p > bid_p_max
        (100, 73, np.random.randint([40]*5, [70]*5), np.random.randint([1]*5, [20]*5)),
        (101, 75, np.random.randint([40]*5, [70]*5), np.random.randint([1]*5, [20]*5)),
        (102, 77, np.random.randint([40]*5, [70]*5), np.random.randint([1]*5, [20]*5)),
        (103, 79, np.random.randint([40]*5, [70]*5), np.random.randint([1]*5, [20]*5)),
        
        # quote_p > bid_p_max > bid_p_min > 1.1*open_p
        (100, 41, np.random.randint([50]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (97, 44, np.random.randint([50]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (94, 53, np.random.randint([60]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        
        # qoute_p > bid_p_max > 1.1*open_p > bid_p_min
        (100, 55, np.array([67, 61, 50, 56, 74]), np.random.randint([1]*5, [20]*5)),
        (102, 52, np.array([86, 73, 74, 53, 50]), np.random.randint([1]*5, [20]*5)),
        (95, 54, np.array([59, 77, 88, 58, 69]), np.random.randint([1]*5, [20]*5)),
        
        # bid_p_max > qoute_p > 1.1*open_p > bid_p_min
        (90, 66, np.array([72, 97, 71, 74, 95]), np.array([2, 19, 14, 12, 10])), # 29
        (87, 68, np.array([92, 71, 74, 90, 77]), np.array([3, 17, 6, 7, 15])), # 10
        
        # bid_p_max > qoute_p > bid_p_min > 1.1*open_p
        (90, 50, np.array([62, 89, 61, 99, 83]), np.array([5, 14, 7, 18, 4])), # 18
        (82, 42, np.array([79, 74, 83, 90, 84]), np.array([4, 13, 14, 3, 16])), # 33
        
        # bid_p_min > qoute_p > 1.1*open_p
        (55, 45, np.array([83, 64, 97, 80, 99]), np.array([16, 16, 6, 5, 6])), # 49
        (55, 44, np.array([80, 62, 95, 95, 90]), np.array([5, 10, 11, 17, 8])), # 51
        
        # bid_p_max > 1.1*open_p > bid_p_min > qoute_p
        (55, 77, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        (52, 75, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        
        # bid_p_min > 1.1*open_p > qoute_p
        (45, 50, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        (39, 57, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        
        # 1.1*open_p > qoute_p > bid_p_max
        (90, 100, np.random.randint([60]*5, [85]*5), np.random.randint([1]*5, [20]*5)),
        (92, 130, np.random.randint([60]*5, [80]*5), np.random.randint([1]*5, [20]*5)),
        
        # 1.1*open_p > bid_p_max > qoute_p > bid_p_min
        (80, 100, np.array([61, 60, 66, 89, 85]), np.random.randint([1]*5, [20]*5)),
        (75, 100, np.array([80, 65, 64, 68, 82]), np.random.randint([1]*5, [20]*5)),
        
        # 1.1*open_p > bid_p_max > bid_p_min > qoute_p
        (55, 100, np.random.randint([60]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (25, 103, np.random.randint([60]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
    ]
    label = [0,0,0,0,0,0,0,0,0,0,29,10,18,33,49,51,0,0,0,0,0,0,0,0,0,0]
    return [train, label]

def get_data6():
    '''
    手工设计的数据集
    '''
    train = [
        # 1
        # quote_p > 0.9*open_p > ask_p_min > ask_p_min
        (100, 105, np.random.randint([60]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (100, 105, np.random.randint([60]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (100, 105, np.random.randint([60]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        # quote_p > ask_p_max > 0.9*open_p > ask_p_min
        (100, 80, np.random.randint([40]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (100, 80, np.random.randint([40]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (100, 80, np.random.randint([40]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        # ask_p_max > quote_p > 0.9*open_p > ask_p_min
        (100, 90, np.random.randint([60]*5, [150]*5), np.random.randint([1]*5, [20]*5)),
        (100, 90, np.random.randint([60]*5, [150]*5), np.random.randint([1]*5, [20]*5)),
        (100, 90, np.random.randint([60]*5, [150]*5), np.random.randint([1]*5, [20]*5)),
        # 2
        # ask_p_max > quote_p > ask_p_min > 0.9*open_p
        (100, 30, np.random.randint([40]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (100, 30, np.random.randint([40]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        (100, 30, np.random.randint([40]*5, [90]*5), np.random.randint([1]*5, [20]*5)),
        # quote_p > ask_p_max > ask_p_min > 0.9*open_p
        (100, 30, np.random.randint([40]*5, [150]*5), np.random.randint([1]*5, [20]*5)),
        (100, 30, np.random.randint([40]*5, [150]*5), np.random.randint([1]*5, [20]*5)),
        (100, 30, np.random.randint([40]*5, [150]*5), np.random.randint([1]*5, [20]*5)),
        # 3
        # ask_p_max > ask_p_min > quote_p > 0.9*open_p
        (50, 30, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        (50, 30, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        (50, 30, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        # 4
        # ask_p_max > ask_p_min > 0.9*open_p > quote_p
        (30, 50, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        (30, 50, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        (30, 50, np.random.randint([60]*5, [100]*5), np.random.randint([1]*5, [20]*5)),
        # 5√
        # ask_p_max > 0.9*open_p > quote_p > ask_p_min
        (60, 80, np.array([129, 44, 134, 69, 116]), np.array([17, 13, 12, 11, 16])), # 13
        (60, 80, np.array([115, 75, 87, 43, 42]), np.array([14, 5, 9, 2, 7])), # 9
        (60, 80, np.array([59, 144, 41, 64, 114]), np.array([10, 14, 5, 11, 4])), # 15
        # 0.9*open_p > quote_p > ask_p_max > ask_p_min
        (80, 100, np.array([42, 43, 72, 66, 62]), np.array([10, 14, 5, 11, 4])), # 44
        (80, 100, np.array([41, 54, 56, 58, 61]), np.array([10, 9, 8, 13, 1])), # 41
        (80, 100, np.array([48, 72, 65, 58, 67]), np.array([3, 2, 9, 7, 8])), # 29
        # 0.9*open_p > ask_p_max > quote_p > ask_p_min
        (64, 100, np.array([75, 74, 42, 46, 69]), np.array([16, 2, 18, 6, 19])), # 24
        (64, 100, np.array([50, 66, 49, 61, 63]), np.array([12, 12, 17, 7, 8])), # 44
        (64, 100, np.array([40, 53, 43, 70, 73]), np.array([19, 10, 4, 15, 3])), # 33
        # 6
        # 0.9*open_p > ask_p_max > ask_p_min > quote_p
        (40, 100, np.random.randint([50]*5, [80]*5), np.random.randint([1]*5, [20]*5)),
        (40, 100, np.random.randint([50]*5, [80]*5), np.random.randint([1]*5, [20]*5)),
        (40, 100, np.random.randint([50]*5, [80]*5), np.random.randint([1]*5, [20]*5)),
        # ask_p_max > 0.9*open_p > ask_p_min > quote_p
        (30, 100, np.random.randint([40]*5, [120]*5), np.random.randint([1]*5, [20]*5)),
        (30, 100, np.random.randint([40]*5, [120]*5), np.random.randint([1]*5, [20]*5)),
        (30, 100, np.random.randint([40]*5, [120]*5), np.random.randint([1]*5, [20]*5)),
    ]
    label = [[0,0],[0,0],[0,0],[0,0],[0,0],
             [0,0],[0,0],[0,0],[0,0],[0,0],
             [0,0],[0,0],[0,0],[0,0],[0,0],
             [0,0],[0,0],[0,0],[0,0],[0,0],
             [0,0],
             [13,72],
             [9,72],
             [15,72],
             [44,90],
             [41,90],
             [29,90],
             [24,90],
             [44,90],
             [33,90],
             [0,0],[0,0],[0,0],[0,0],[0,0],[0,0]
             ]
    return [train, label]

def get_data7():
    '''
    hold_list测试数据集
    '''
    train, label = [], []
    dim = 5
    for i in range(50):
        train.append((np.random.randint([0]*dim, [2]*dim),
                        np.random.randint([1]*dim, [20]*dim),
                        np.random.randint([20]*dim, [45]*dim),
                        np.random.randint(25, 40)))
    for i in range(50):
        side, quantity, price, open_p = train[i]
        label.append((side, quantity*(1-LimUp_(price, open_p))*(1-LimDown_(price, open_p)), price*(1-LimUp_(price, open_p))*(1-LimDown_(price, open_p))))
    return [train, label]

In [73]:
# 涨跌判断器
# fs = [
#         # Function(op.add, 2), 
#         Function(op.sub, 2), 
#         # Function(op.mul, 2), 
#         # Function(protected_div, 2),
#         Function(op.abs, 1),
#         Function(sign, 1),
#         Function(tenth, 1),
#     ]

# 三元求和器
# fs = [
#     Function(op.ge, 2, '≥'),
#     # Function(Sub, 2, name='—'),
#     # Function(Sign, 1, name='sgn'),
#     Function(Mul, 2, name='x'),
#     Function(Sum, 1, name='∑')
# ]


# 涨停处理
# fs = [
#     Function(Sum3, 3, name='Sum3'),
#     Function(LimUp, 2, name='LimUp'),
#     Function(MAX, 1, name='MAX'),
#     Function(Sub, 2, name='—'),
#     Function(Max1, 1, name='Max1'),
#     Function(Mul, 2 , name='x'),
# ]
# 跌停处理,连带输出价格
# fs = [
#     Function(Sum3_, 3, name='Sum3'),
#     Function(LimDown, 2, name='LimDown'),
#     Function(MAX, 1, name='MAX'),
#     Function(Sub, 2, name='—'),
#     Function(Max1, 1, name='Max1'),
#     Function(Mul, 2 , name='x'),
#     Function(x09, 1, name='×0.9'),
# ]

# holdlist
def XOR(x): return 1-x
def equal(x): return x
fs = [
    Function(LimUp_, 2, name='LimUp'),
    Function(LimDown_, 2, name='LimDown'),
    Function(Mul, 2, name='×'),
    Function(XOR, 1, name='1-'),
    Function(equal, 1, name='×1')
]
def Abs(x): return np.abs(x)
new_functions = [
    # Function(Add, 2, name='Add'),
    Function(Sub, 2, name='Sub'),
    # Function(Mul, 2, name='Mul'),
    # Function(Div, 2, name='Div'),
    # Function(Neg, 1, name='Neg'),
    Function(Abs, 1, name='Abs'),
    # Function(Max1, 1, name='Max2'),
    # Function(Sum, 1, name='Sum'),
    # Function(Min1, 1, name='Min'),
    Function(Sign, 1, name='Sgn'),
    Function(x01, 1,)
    # Function(const_01, 0, name='0.1')
]

In [74]:
class Node:
    """A node in CGP graph"""
    def __init__(self, arity):
        """Initialize this node randomly"""
        self.arity = arity
        self.i_func = None # 该节点的函数在函数集的index
        self.i_inputs = [None] * arity
        self.weights = [None] * arity
        self.i_output = None
        self.output = None
        self.active = False

class Individual:
    """An individual (chromosome, genotype, etc.) in evolution"""
    
    # 以下是class变量，init内是实例化object的变量
    # max_arity = 3
    # fitness = None

    def __init__(self, input_dim, out_dim, function_set, n_cols, level_back, out_random_active=False):
        
        self.n_cols = n_cols # number of cols (nodes) in a single-row CGP
        self.level_back = level_back # 后面的节点可以最远连接的前面节点的相对位置
    
        # 【创新点：给不同位置的node设置不同的function set,人的先验知识可以起作用，可能需要给node class设置一个fun_set】
        self.function_set = function_set
        self.max_arity = max(f.arity for f in fs)
        self.n_inputs = input_dim
        self.n_outputs = out_dim # 输出维度
        self.weight_range = [-1, 1]
        
        # 创建nodes
        self.nodes = []
        for pos in range(self.n_cols):
            self.nodes.append(self._create_random_node(pos))
        
        # 将最后n_outputs个node设为输出节点
        for i in range(1, self.n_outputs + 1):
            # 输出节点有一定概率被关闭
            if out_random_active:
                if random.random() < 0.8:
                    self.nodes[-i].active = True
                else:
                    self.nodes[-i].active = False
            else:
                self.nodes[-i].active = True
        
        self.fitness = None
        self._active_determined = False

    def _create_random_node(self, pos):
        '''
        pos:该节点的index
        设: n_inputs=3, level_back=4

        in  in  in  0   1   2   3   4   5   6
        *   *   *   *   *   *   *   *   *   *

        pos  pos-level_back  -n_inputs  max(p-l,-n)  pos-1  i_inputs取值
        0        -4              -3         -3         -1    -3,-2,-1
        1        -3              -3         -3          0    -3,-2,-1,0
        2        -2              -3         -2          1    -2,-1,0,1
        3        -1              -3         -1          2    -1,0,1,2
        4         0              -3          0          3     0,1,2,3
        5         1              -3          1          4     1,2,3,4
        6         2              -3          2          5     2,3,4,5
        
        输入维度=3,则-3,-2,-1三个点是程序的输入节点
        '''
        node = Node(self.max_arity)
        node.i_func = random.randint(0, len(self.function_set) - 1)
        for i in range(self.function_set[node.i_func].arity):
            # 随机确定node的每个输入端口连接的是前面节点(column)的输出
            # node.i_inputs[i]记录前端父节点的idx
            # print('aaa', i, self.function_set[node.i_func].name)
            node.i_inputs[i] = random.randint(max(pos - self.level_back, -self.n_inputs), pos - 1)
            node.weights[i] = 1.0 # random.uniform(self.weight_range[0], self.weight_range[1])
        node.i_output = pos
        return node

    def _determine_active_nodes(self):
        """
        Determine which nodes in the CGP graph are active
        """
        # check each node in reverse order
        n_active = 0

        # 逆序遍历所有节点
        for node in reversed(self.nodes):
            if node.active:
                n_active += 1
                # 依次检查该节点所有输入端口
                for i in range(self.function_set[node.i_func].arity):
                    
                    # i_input是该node第i的输入端口所连的父节点的index
                    i_input = node.i_inputs[i]
                    
                    if i_input >= 0:  # >=0表示node的父节点是一个hidden节点，而非input节点
                        # 该节点的父节点也设置为“激活”
                        self.nodes[i_input].active = True
        self.n_active = n_active
        if config.Verbose:
            print("# active genes: ", n_active)

    def eval(self, *args):
        """
        Given inputs, evaluate the output of this CGP individual.
        :return the final output value
        """
        if not self._active_determined:
            self._determine_active_nodes()
            self._active_determined = True
        
        # forward pass: evaluate
        for node in self.nodes:
            if node.active:
                inputs = []
                for i in range(self.function_set[node.i_func].arity): # 依次获得该node各维输入
                    i_input = node.i_inputs[i] # node的父节点idx
                    w = node.weights[i] # node与父节点连边的权重
                    
                    # 父节点是程序的输入节点
                    if i_input < 0:
                        inputs.append(args[-i_input - 1] * w)
                    # 父节点是普通节点
                    else:
                        inputs.append(self.nodes[i_input].output * w)
                node.output = self.function_set[node.i_func](*inputs) # 执行计算
        if self.n_outputs == 1:
            return self.nodes[-1].output
        
        out = []
        for i in reversed(range(1, self.n_outputs + 1)):
            out.append(self.nodes[-i].output)
        return out

    def mutate(self, mut_rate=0.01):
        """
        Mutate this individual. Each gene is varied with probability *mut_rate*.
        :param mut_rate: mutation probability
        :return a child after mutation
        """
        child = copy.deepcopy(self)
        for pos, node in enumerate(child.nodes):
            # mutate the function gene
            if random.random() < mut_rate:
                node.i_func = random.choice(range(len(self.function_set)))
            
            # 函数突变之后需要重新选择其父节点
            # mutate the input genes (connection genes)
            arity = self.function_set[node.i_func].arity
            for i in range(arity):
                if node.i_inputs[i] is None or random.random() < mut_rate:  # if the mutated function requires more arguments, then the last ones are None 
                    node.i_inputs[i] = random.randint(max(pos - self.level_back, -self.n_inputs), pos - 1)
                if node.weights[i] is None or random.random() < mut_rate:
                    # 节点之间不再设置随机权重，统一固定1.0
                    node.weights[i] = 1.0 # random.uniform(self.weight_range[0], self.weight_range[1])
            # initially an individual is not active except the last output node
            node.active = False
        for i in range(1, self.n_outputs + 1):
            child.nodes[-i].active = True
        child.fitness = None
        child._active_determined = False
        return child

# Individual.function_set = fs
# Individual.max_arity = max(f.arity for f in fs)

def evolve(pop, mut_rate, mu, lambda_):
    """
    Evolve the population *pop* using the mu + lambda evolutionary strategy

    :param pop: a list of individuals, whose size is mu + lambda. The first mu ones are previous parents.
    :param mut_rate: mutation rate
    :return: a new generation of individuals of the same size
    """
    pop = sorted(pop, key=lambda ind: ind.fitness)  # stable sorting
    parents = pop[-mu:]
    # generate lambda new children via mutation
    offspring = []
    for _ in range(lambda_):
        parent = random.choice(parents)
        offspring.append(parent.mutate(mut_rate))
    return parents + offspring


def create_population(n, input_dim, out_dim, fs, n_cols, level_back, out_random_active=False):
    """Create a random population composed of n individuals."""
    return [Individual(input_dim, 
                       out_dim, 
                       fs, 
                       n_cols,
                       level_back,
                       out_random_active=out_random_active) for _ in range(n)]


In [75]:
np.random.seed(int(time.time()))
random.seed(int(time.time()))
pop = create_population(config.MU+config.LAMBDA, 
                            input_dim=4, 
                            out_dim=3,
                            fs=fs,
                            # fs=new_functions,
                            n_cols=8,
                            level_back=8,
                            out_random_active=False)
# data = get_data()
# data = get_data2()
# data = get_data3()
# data = get_data5()
# data = get_data6()
data = get_data7()

In [67]:
for g in range(1000000000):
    tick = time.time()
    fit_list = [rollout5(data, p) for p in pop]
    # fitness = ray.get(fit_list)
    # 优化方向是最大化
    for f, p in zip(fit_list, pop):
        p.fitness = f
    pop = evolve(pop, 0.15, config.MU, config.LAMBDA)
    print(g, 'time:', round(time.time()-tick, 3),'best fitness:', pop[0].fitness)
    if pop[0].fitness == 1.0:
        break

0 time: 0.16 best fitness: 0.25
1 time: 0.117 best fitness: 0.25
2 time: 0.138 best fitness: 0.25
3 time: 0.146 best fitness: 0.25
4 time: 0.158 best fitness: 0.25
5 time: 0.248 best fitness: 0.25
6 time: 0.179 best fitness: 0.25
7 time: 0.147 best fitness: 0.25
8 time: 0.141 best fitness: 0.25
9 time: 0.159 best fitness: 0.25
10 time: 0.137 best fitness: 0.25
11 time: 0.15 best fitness: 0.25
12 time: 0.129 best fitness: 0.25
13 time: 0.129 best fitness: 0.25
14 time: 0.119 best fitness: 0.25
15 time: 0.114 best fitness: 0.25
16 time: 0.105 best fitness: 0.25
17 time: 0.1 best fitness: 0.25
18 time: 0.139 best fitness: 0.25
19 time: 0.139 best fitness: 0.25
20 time: 0.147 best fitness: 0.25
21 time: 0.152 best fitness: 0.25
22 time: 0.135 best fitness: 0.25
23 time: 0.132 best fitness: 0.25
24 time: 0.138 best fitness: 0.25
25 time: 0.128 best fitness: 0.25
26 time: 0.133 best fitness: 0.25
27 time: 0.16 best fitness: 0.25
28 time: 0.2 best fitness: 0.25
29 time: 0.18 best fitness: 0.2

KeyboardInterrupt: 

In [68]:
g = extract_computational_subgraph(pop[0])
visualize(g, to_file='test.png',input_names=['side', 'quantity', 'price', 'open_p']) # ['quote_p', 'open_p', 'ask_p', 'ask_q']

Operator notation of '×1'' is not available. The node id is shown instead.
Operator notation of '×1'' is not available. The node id is shown instead.
Operator notation of '×1'' is not available. The node id is shown instead.
Operator notation of 'LimDown'' is not available. The node id is shown instead.
Operator notation of '×'' is not available. The node id is shown instead.


In [5]:
class Individual:
    def __init__(self, input_dim, out_dim, function_set, n_cols, level_back, out_random_active=False):
        self.n_cols = n_cols # number of cols (nodes) in a single-row CGP
        self.level_back = level_back # 后面的节点可以最远连接的前面节点的相对位置
        self.function_set = function_set
        self.max_arity = max(f.arity for f in fs)
        self.n_inputs = input_dim
        self.n_outputs = out_dim # 输出维度
        self.weight_range = [-1, 1]
        
        # 创建nodes
        self.nodes = []
        for pos in range(self.n_cols):
            self.nodes.append(self._create_random_node(pos))
        
        # 将最后n_outputs个node设为输出节点
        for i in range(1, self.n_outputs + 1):
            if out_random_active: # 输出节点有一定概率被关闭
                if random.random() < 0.8: self.nodes[-i].active = True
                else: self.nodes[-i].active = False
            else: self.nodes[-i].active = True
        
        self.fitness = None
        self._active_determined = False

    def _create_random_node(self, pos):
        node = Node(self.max_arity)
        node.i_func = random.randint(0, len(self.function_set) - 1)
        for i in range(self.function_set[node.i_func].arity):
            # 随机确定node的每个输入端口连接的是前面节点(column)的输出
            # node.i_inputs[i]记录前端父节点的idx
            node.i_inputs[i] = random.randint(max(pos - self.level_back, -self.n_inputs), pos - 1)
            node.weights[i] = 1.0 # random.uniform(self.weight_range[0], self.weight_range[1])
        node.i_output = pos
        return node

    def _determine_active_nodes(self):
        """
        Determine which nodes in the CGP graph are active
        """
        n_active = 0
        for node in reversed(self.nodes):
            if node.active:
                n_active += 1
                for i in range(self.function_set[node.i_func].arity):
                    i_input = node.i_inputs[i]
                    if i_input >= 0: self.nodes[i_input].active = True
        self.n_active = n_active
        if config.Verbose: print("# active genes: ", n_active)

    def eval(self, *args):
        """
        Given inputs, evaluate the output of this CGP individual.
        :return the final output value
        """
        if not self._active_determined:
            self._determine_active_nodes()
            self._active_determined = True
        print('aaa',args)
        # forward pass: evaluate
        for node in self.nodes:
            if node.active:
                inputs = []
                for i in range(self.function_set[node.i_func].arity): # 依次获得该node各维输入
                    i_input = node.i_inputs[i] # node的父节点idx
                    w = node.weights[i]
                    if i_input < 0:
                        inputs.append(args[-i_input - 1] * w)
                    else:
                        inputs.append(self.nodes[i_input].output * w)
                print('bbb', '\n',
                        self.function_set[node.i_func].name, '\n',
                        node.i_inputs,  '\n',
                        inputs)
                node.output = self.function_set[node.i_func](*inputs) # 执行计算
        if self.n_outputs == 1:
            return self.nodes[-1].output
        out = []
        for i in reversed(range(1, self.n_outputs + 1)):
            out.append(self.nodes[-i].output)
        return out

    def mutate(self, mut_rate=0.01):
        child = copy.deepcopy(self)
        for pos, node in enumerate(child.nodes):
            if random.random() < mut_rate:
                node.i_func = random.choice(range(len(self.function_set)))
            arity = self.function_set[node.i_func].arity
            for i in range(arity):
                if node.i_inputs[i] is None or random.random() < mut_rate:  # if the mutated function requires more arguments, then the last ones are None 
                    node.i_inputs[i] = random.randint(max(pos - self.level_back, -self.n_inputs), pos - 1)
                if node.weights[i] is None or random.random() < mut_rate:
                    node.weights[i] = 1.0
            node.active = False
        for i in range(1, self.n_outputs + 1):
            child.nodes[-i].active = True
        child.fitness = None
        child._active_determined = False
        return child

In [8]:
ind = Individual(2,1,fs,3,3,)
ind.nodes[0].i_func = 0
ind.nodes[0].i_inputs = [-1, -2]
ind.nodes[1].i_func = 1
ind.nodes[1].i_inputs = [-1, 0]
ind.nodes[2].i_func = 2
ind.nodes[2].i_inputs = [1, None]
for node in ind.nodes:
    print(fs[node.i_func].name, node.i_inputs)
data = get_data3()
train, label = data
count = 0
for xy, l in zip(train, label):
    output = ind.eval(*xy)
    print('ccc', output, l)
    if not type(output)==np.ndarray:
        # print(output, l)
        if output == l:
            count += 1
print(count / 100)

≥ [-1, -2]
x [-1, 0]
∑ [1, None]
aaa (array([-12,   5,   4,  -2,  16,  17, -14,  -5, -15,  19]), -3)
bbb 
 ≥ 
 [-1, -2] 
 [array([-12.,   5.,   4.,  -2.,  16.,  17., -14.,  -5., -15.,  19.]), -3.0]


TypeError: unsupported operand type(s) for *: 'bool' and 'NoneType'